In [1]:
import numpy as n
from scipy.optimize import root
import Properties as prop
import pandas as pd

calctolerance=10**-5
calcmethod = "hybr"
KPD=0.99

#Таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#Таблица частичной нагрузки (входные данные)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

class Heatex:
    def __init__(self, stream11, stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P021 = water_streams0.at[stream21,'P']
        self.P022 = water_streams0.at[stream22,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.G2   = water_streams.at[stream21,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P21  = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P021)
        T022   = prop.water_T_HP(self.H022,self.P022)
        dTmin0 = min(T011-T022,T012-T021)
        dTmax0 = max(T011-T022,T012-T021)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        T02av  = (T021+T022)/2
        P02av  = (self.P021+self.P022)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        ro02av = prop.water_D_PT(P02av,T02av)
        ro021  = prop.water_DSS_P(self.P021)
        #Потери давления по воде (ro по входу!):
        ro21   = prop.water_DSS_P(self.P21)
        ddp    = (ro21/ro021)*((self.G02/self.G2)**2)
        P22    = self.P21 - (self.P021-self.P022)/ddp
        P12    = self.P1
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P21)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = self.H21 + (Q/self.G2)
            T22 = prop.water_T_HP(H22,P22)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q*100)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        
        H22 = self.H21 + (Q/self.G2)
        T22 = prop.water_T_HP(H22,P22)
        gas_streams.loc[stream12,'T':'G'] = [T12,P12,H12,self.G1]
        gas_streams.loc[stream12,'N2':'Ar'] = [gas_streams.at[stream11,'N2'],gas_streams.at[stream11,'O2'],gas_streams.at[stream11,'CO2'],gas_streams.at[stream11,'H2O'],gas_streams.at[stream11,'Ar']]
        water_streams.loc[stream22,'T':'G'] = [T22,P22,H22,self.G2]
        return [T12,P12,H12,self.G1,T22,P22,H22,self.G2,Q]

stream11 = 'GTU-PEVD'
stream12 = 'PEVD-IVD'
stream21 = 'IVD-PEVD'
stream22 = 'PEVD-DROS'

import time
start_time = time.time()

PEVD=Heatex(stream11, stream12,stream21,stream22).calc()
water_streams

,T,P,H,G
PEVD-DROS,529.637207,8.711142,3464.724387,20.000000
IVD-PEVD,301.297556,8.746000,2747.104023,20.000000
EVD-IVD,298.297556,8.746000,1335.087928,62.370779
